# Project

## Dépendance

In [1]:
! pip install SPARQLWrapper
! pip install exifread
! pip install pandas
! pip install PIL
! pip install ipywidgets
! pip install -U scikit-learn

DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


## Collecte de données

In [2]:
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 15"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

,ville,pays,image
0,Pskov,Russie,http://commons.wikimedia.org/wiki/Special:File...
1,Copenhague,Danemark,http://commons.wikimedia.org/wiki/Special:File...
2,Québec,Canada,http://commons.wikimedia.org/wiki/Special:File...
3,Pune,Inde,http://commons.wikimedia.org/wiki/Special:File...
4,Riazan,Russie,http://commons.wikimedia.org/wiki/Special:File...
5,Erfurt,Allemagne,http://commons.wikimedia.org/wiki/Special:File...
6,Djouba,Soudan du Sud,http://commons.wikimedia.org/wiki/Special:File...
7,Edmonton,Canada,http://commons.wikimedia.org/wiki/Special:File...
8,Victoria,Canada,http://commons.wikimedia.org/wiki/Special:File...
9,Francfort-sur-le-Main,Allemagne,http://commons.wikimedia.org/wiki/Special:File...


### Téléchargement des images

In [3]:
import requests
import shutil
import os

def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        # Create 'images' directory if it doesn't exist
        if not os.path.exists("images"):
            os.makedirs("images")

        # Extract the filename from the URL and save the image in 'images' directory
        filename = os.path.join("images", os.path.basename(url))
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

def clear_images_directory():
    # Clear 'images' directory if it exists
    if os.path.exists("images"):
        shutil.rmtree("images")

# Clear 'images' directory before every execution
clear_images_directory()

# Assuming 'dataframe' is a DataFrame containing URLs under column 'image'
# Apply the download_image function to each URL in the 'image' column
dataframe.image.apply(download_image)

0     200
1     200
2     200
3     200
4     200
5     200
6     200
7     200
8     200
9     200
10    200
11    200
12    200
13    200
14    200
Name: image, dtype: int64

### Enregistrement des métadonnées
#### Import

In [4]:
import os
import json
from PIL import Image, TiffImagePlugin
import PIL.ExifTags
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

#### Define the directory path and initialize the list to store metadata:

In [5]:
directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

#### Define the function to cast data types

In [6]:
def cast(v):
    if isinstance(v, TiffImagePlugin.IFDRational):
        if v.denominator == 0:
            return None  # Handle division by zero gracefully
        return float(v.numerator) / float(v.denominator)
    elif isinstance(v, tuple):
        return tuple(cast(t) for t in v)
    elif isinstance(v, bytes):
        return v.decode(errors="replace")
    elif isinstance(v, dict):
        for kk, vv in v.items():
            v[kk] = cast(vv)
        return v
    else:
        return v

#### Define the function to get main colors

In [7]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

def get_main_colors(image_path, num_clusters):
    # Open and resize the image
    imgfile = image_path
    imgfile.thumbnail((100, 100))  # Resize the image to a smaller size
    numarray = np.array(imgfile)  # Convert image to numpy array

    # Flatten the image array
    numarray = numarray.reshape((-1, 3))

    # Sample a subset of pixels (optional)
    np.random.shuffle(numarray)
    num_samples = min(10000, len(numarray))  # Adjust the number of samples if needed
    numarray = numarray[:num_samples]

    # Perform KMeans clustering
    clusters = KMeans(n_clusters=num_clusters, n_init=2)
    clusters.fit(numarray)

    # Get the main colors
    main_colors = clusters.cluster_centers_.astype(int)

    return main_colors.tolist()  # Convert array to list


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import numpy as np

# Function to find the closest main color for a given RGB value
def find_closest_color(rgb):
    main_colors = [
        [0, 0, 0],      # Black
        [255, 255, 255],  # White
        [255, 0, 0],    # Red
        [0, 255, 0],    # Green
        [0, 0, 255],    # Blue
        [255, 255, 0],  # Yellow
        [255, 0, 255],  # Magenta
        [0, 255, 255],  # Cyan
        [128, 128, 128], # Grey
        [255, 165, 0],   # Orange
        [0, 128, 0],     # Dark Green
        [128, 0, 128],   # Purple
        [128, 0, 0],     # Maroon
        [0, 128, 128],   # Teal
        [128, 128, 0],   # Olive
        [0, 255, 255],   # Light Blue
        [210, 105, 30],  # Chocolate
        [139, 69, 19],   # Saddle Brown
        [255, 192, 203], # Pink
        [0, 0, 128],     # Navy
        [220, 20, 60]    # Crimson
        # Add more main colors as needed
    ]

    # Convert list of main colors to numpy array for vectorized calculations
    main_colors = np.array(main_colors)

    # Convert input RGB list to numpy array
    rgb = np.array(rgb)

    # Calculate Euclidean distances between input RGB and all main colors
    distances = np.linalg.norm(main_colors - rgb, axis=1)

    # Find index of the closest main color
    closest_index = np.argmin(distances)

    return main_colors[closest_index].tolist()  # Convert NumPy array to list


#### Define the function to get the images properties

In [ ]:
def get_image_metadata(imgfile, hasExif):
    metadata = {}

    # Get image format
    metadata['format'] = imgfile.format

    # Get image size
    metadata['size'] = imgfile.size
    
    # Get 3 main colors
    main_colors = get_main_colors(imgfile, 3)
    metadata["main_colors"] = main_colors
    
    # Get closest colors based on the 3 main colors
    closest_colors = [find_closest_color(color) for color in main_colors]
    
    metadata["closest_colors"] = closest_colors
    
    if hasExif:
        exif = dict(imgfile._getexif().items())
        if exif:
            # Get image orientation (landscape, portrait, square, etc.)
            if 274 in exif:
                orientation = exif[274]
                if orientation == 1:
                    metadata['orientation'] = 'Landscape'
                elif orientation == 3:
                    metadata['orientation'] = 'Portrait'
                else:
                    metadata['orientation'] = 'Unknown'
            else:
                metadata['orientation'] = 'Unknown'
    else:
        metadata['orientation'] = 'Unknown'
    return metadata


#### Loop through files in the directory and extract metadata

It will also add as tags the format, the size and the 3 main colors

In [ ]:
import os
import json
import sys
from PIL import Image, TiffImagePlugin
import PIL.ExifTags

directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)) and filename.lower().endswith((".jpg", ".png")):
        print(f"Found image: {filename}")

        try:
            imgfile = Image.open(os.path.join(directory_path, filename))
            exif_data = imgfile._getexif()
            
            metadata_dict = {}
            
            hasExif = False

            if exif_data:
                hasExif = True
                for k, v in imgfile._getexif().items():
                    if k in PIL.ExifTags.TAGS:
                        v = cast(v)
                        metadata_dict[PIL.ExifTags.TAGS[k]] = v
            else:
                print(f"  - No EXIF data found for {filename}")

            # Get additional image metadata (format, size, orientation)
            image_metadata = get_image_metadata(imgfile, hasExif)
            metadata_dict.update(image_metadata)
            
            all_metadata.append({filename: metadata_dict})
            if not metadata_dict:
                print(f"  - No metadata found for {filename}")

        except Exception as e:
            print(f"  - Error processing {filename}: {e}")

directory_path = "output"
output_file = "directory_metadata.json"

if not os.path.exists(directory_path):
            os.makedirs(directory_path)

output_location = directory_path + "/" + output_file
with open(output_location, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Directory metadata saved to {output_location} (if no errors occurred)")


In [ ]:
import json
import matplotlib.pyplot as plt

# Open the JSON file
with open('output/directory_metadata.json', 'r') as file:
    # Load the JSON data
    json_data = json.load(file)

    # Create a list to store all the closest colors
    all_closest_colors = []

    # Iterate through each object in the list
    for obj in json_data:
        # Get the dictionary within each object
        for filename, details in obj.items():
            # Access the "closest_colors" key within each dictionary
            closest_colors = details.get("closest_colors")
            if closest_colors:
                # Extend the list of all closest colors with the closest colors of the current image
                all_closest_colors.extend(closest_colors)

# Count the occurrences of each closest color
unique_closest_colors, counts = np.unique(all_closest_colors, axis=0, return_counts=True)

# Calculate percentages
total_count = sum(counts)
percentages = [(count / total_count) * 100 for count in counts]

# Create labels with RGB values and percentages
labels = [f"RGB: {color}, {percentage:.2f}%" for color, percentage in zip(unique_closest_colors, percentages)]
sizes = counts
colors = [f"#{r:02x}{g:02x}{b:02x}" for r, g, b in unique_closest_colors]

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Pie Chart of Closest Colors')
plt.show()


## Étiquetage et annotation
### Utilisation des utilisateur pour tagger les images

In [ ]:
from os import listdir
import ipywidgets as widgets
from ipywidgets import GridspecLayout

images = []

# Loop through files in the "./images" directory
for file in listdir("./images"):
    # Check if the file is a PNG or JPG image
    if file.endswith((".png", ".jpg")):
        images.append("./images/" + file)
        
values = []

# Create text boxes
text_boxes = [widgets.Text(value='', description='Tag:') for _ in range(len(images))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(images), width='400px')
for i, (img, text_box) in enumerate(zip(images, text_boxes)):
    with open(img, "rb") as file:
        image = file.read()
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=100,
        height=100,
    )
    layout[i,0] = image_widget
    layout[i, 1] = text_box

# Button to get selected images
button = widgets.Button(description="Select")

# Output widget to display selected values
output = widgets.Output()

# Function to get selected values
def get_selected_values(btn):
    global values
    values = []
    selected_values = [(file.split("/")[-1], text_box.value) for file, text_box in zip(images, text_boxes) if text_box.value]
    with output:
        output.clear_output()
        print("Selected Values:")
        for file_name, value in selected_values:
            print(f"{file_name}: {value}")
            values.append((file_name, value))
    print(values)
        

# Link button click event to function
button.on_click(get_selected_values)

# Display the layout and button
display(layout, button, output)

In [ ]:
import json

# Load the existing JSON file
with open('output/directory_metadata.json', 'r') as file:
    json_data = json.load(file)

# Iterate through the values and update the JSON data
for file_name, value in values:
    # Search for the corresponding object in the JSON data
    for obj in json_data:
        for filename, details in obj.items():
            # Check if the file name matches
            if filename == file_name:
                # Update the value
                details['tag'] = value
                break  # Exit inner loop if match found
        else:
            continue  # Continue to next iteration if no match found
        break  # Exit outer loop if match found

# Save the updated JSON data back to the file
with open('output/directory_metadata.json', 'w') as file:
    json.dump(json_data, file, indent=4)